# Collect data
# Cleanup data
# WordCloud
# Sentiment (Vader) -> check rougness of lyrics per year/ check number of abcense words
# topic modelling
# Web wiki db -> plot map of relation between birth place and artists in top rank
# Train NN T5 ->< try to explain lyrics -> explain all text

In [2]:
!pip install torch --quiet
!pip install sklearn --quiet
!pip install datasets --quiet
!apt install git-lfs --quiet
!pip install transformers --quiet
!pip install sentencepiece --quiet


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install pandas --quiet
!pip install numpy --quiet


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install beautifulsoup4 --quiet
!pip install lyricsgenius --quiet
!pip install billboard.py


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# !pip install git+https://github.com/johnwmillr/LyricsGenius.git --quiet

In [5]:
!pip install python-dotenv
from dotenv import load_dotenv
import dotenv
import os
%load_ext dotenv
%dotenv


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [6]:
# Import Genious access token

In [7]:
load_dotenv(os.path.join(os.getcwd(),'.env'))
GENIOUS_ACCESS_TOKEN = os.environ.get('GENIOUS_ACCESS_TOKEN')

In [8]:
import pandas as pd
import numpy as np
import requests

# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns
from IPython.display import HTML, display
from bs4 import BeautifulSoup
# from nltk.tokenize import RegexpTokenizer
import lyricsgenius as genius
import billboard
import sys
import re
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# import string
# from nltk.stem import PorterStemmer
from datetime import datetime, date, timedelta
# from wordcloud import WordCloud
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# import spacy
# from collections import Counter
# from os import path
# from PIL import Image
# from keras.models import model_from_json
# import pickle


import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
from huggingface_hub import notebook_login
import os
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None
display(HTML("<style>.container { width:100% !important; }</style>"))
% config InlineBackend.figure_format = 'png'

UsageError: Line magic function `%` not found.


In [ ]:
def disable_pandas_warnings():
    import warnings
    warnings.resetwarnings()  # Maybe somebody else is messing with the warnings system?
    warnings.filterwarnings('ignore')  # Ignore everything
    # ignore everything does not work: ignore specific messages, using regex
    warnings.filterwarnings('ignore', '.*A value is trying to be set on a copy of a slice from a DataFrame.*')
    warnings.filterwarnings('ignore', '.*indexing past lexsort depth may impact performance*')

In [ ]:
disable_pandas_warnings()

## Data Collecting

In [23]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days // 7)):
        yield start_date + timedelta(n*7)

In [24]:
def collect_songs_from_billboard_each_day(start_date_str, end_date_str, chart_name='hot-100', table_path = None,to_csv = False):
    start_date = datetime.strptime(
        start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    columns = ["Rank",
                    "Song Title",
                    "Artist",
                    "Date",
                    "Year"]
    dataset = pd.DataFrame(columns = columns, data = [])
    if to_csv:
        dataset.to_csv(table_path,index=False)

    sys.stdout.write(
        "\r" + "Collecting Songs from " + start_date_str + " to " + end_date_str + " via https://www.billboard.com")
    sys.stdout.flush()
    for single_date in tqdm(daterange(start_date, end_date)):
        print(single_date.strftime('%Y-%m-%d'))
        chart = billboard.ChartData(chart_name, date=single_date.strftime('%Y-%m-%d'))
        for song in chart:
            row = {
                "Rank": song.rank,
                "Song Title": song.title,
                "Artist": song.artist,
                "Date": single_date.strftime('%Y-%m-%d'),
                "Year": single_date.strftime('%Y')
            }
            if to_csv:
                pd.DataFrame([row]).to_csv(table_path, mode='a',index=False, header=False)
            else:
                dataset = dataset.append(row, ignore_index=True)  #pd.concat([dataset,pd.DataFrame(row)],ignore_index=True)#
    if not to_csv:
        return dataset


In [ ]:
all_songs = collect_songs_from_billboard_each_day('2003-02-17', '2023-02-17',table_path='all_songs_20.csv',to_csv = True)

In [4]:
all_songs = pd.read_csv("all_songs_20.csv")

In [5]:
all_songs.head(70)

,Rank,Song Title,Artist,Date,Year
0,1,All I Have,Jennifer Lopez Featuring LL Cool J,2003-02-17,2003
1,2,In Da Club,50 Cent,2003-02-17,2003
2,3,Mesmerize,Ja Rule Featuring Ashanti,2003-02-17,2003
3,4,"Bump, Bump, Bump",B2K & P. Diddy,2003-02-17,2003
4,5,Cry Me A River,Justin Timberlake,2003-02-17,2003
...,...,...,...,...,...
65,66,No Letting Go,Wayne Wonder,2003-02-17,2003
66,67,Make It Clap,Busta Rhymes Featuring Spliff Star,2003-02-17,2003
67,68,Can't Stop,Red Hot Chili Peppers,2003-02-17,2003
68,69,What Happened To That Boy,Baby Featuring Clipse,2003-02-17,2003


In [6]:
all_songs["Artist"][all_songs['Artist'] == "Jackson 5"] = "The Jackson 5"
all_songs["Artist"][all_songs['Artist'] == "Beatles"] = "The Beatles"

In [8]:
all_songs.drop_duplicates(subset=['Artist','Song Title']).to_csv('unique_songs_20.csv', mode='a',index=False)

In [9]:
api = genius.Genius(GENIOUS_ACCESS_TOKEN,timeout = 5, sleep_time=0.015, verbose=False)

In [10]:
all_songs = pd.read_csv("unique_songs_20.csv")

GENIOUS_CSV = 'all_songs_data_new.csv'
columns = [ "Year",
        "Rank",
        "Song Title",
        "Artist",
        "Primary Artists",
        "Lyrics",
        "Song URL",
        "Song id",
        "Song stats",
        "Song annotations"]
dataset = pd.DataFrame(columns = columns, data = [])
dataset.to_csv(GENIOUS_CSV,index=False)


all_song_data = pd.DataFrame()
start_time = datetime.now()
print("Started at {}".format(start_time))
for i in range(0, len(all_songs)):
    rolling_pct = int((i / len(all_songs)) * 100)
    # print(str(rolling_pct) + "% complete." + " Collecting Record " + str(i) + " of " +
    #       str(len(all_songs)) + ". Year " + str(all_songs.iloc[i]['Year']) + "." + " Currently collecting " +
    #       all_songs.iloc[i]['Song Title'] + " by " + all_songs.iloc[i]['Artist'] + " " * 50, end="\r")
    song_title = all_songs.iloc[i]['Song Title']
    song_title = re.sub(" and ", " & ", song_title)
    artist_name = all_songs.iloc[i]['Artist']
    artist_name = re.sub(" and ", " & ", artist_name)

    try:
        song = api.search_song(song_title, artist=artist_name)
        primary_artists = song.primary_artist
        song_lyrics = re.sub("\n", " ", song.lyrics)  #Remove newline breaks, we won't need them.
        song_url = song.url
        song_id = song.id
        song_stats = song.stats
        song_annotations = api.song_annotations(song.id)
    except:
        primary_artists = "null"
        song_lyrics = "null"
        song_url = "null"
        song_id = "null"
        song_stats = 'null'
        song_annotations = 'null'

    row = {
        "Year": all_songs.iloc[i]['Year'],
        "Rank": all_songs.iloc[i]['Rank'],
        "Song Title": all_songs.iloc[i]['Song Title'],
        "Artist": all_songs.iloc[i]['Artist'],
        "Primary Artists": primary_artists,
        "Lyrics": song_lyrics,
        "Song URL": song_url,
        "Song id": song_id,
        "Song stats": song_stats,
        "Song annotations": song_annotations
    }
    pd.DataFrame([row]).to_csv(GENIOUS_CSV, mode='a',index=False, header=False)
    # all_song_data = all_song_data.append(row, ignore_index=True)
end_time = datetime.now()
print("\nCompleted at {}".format(start_time))
print("Total time to collect: {}".format(end_time - start_time))


Started at 2023-03-18 15:38:32.176995

Completed at 2023-03-18 15:38:32.176995
Total time to collect: 5:20:57.880854


In [47]:
song = api.search_song('THE STORY OF ALISHER', artist='Oxxxymiron')

In [48]:
# for i in song.to_dict():
print(api.song_annotations(song.id))
    # print(api.song_comments(song.to_dict()['id']))

[('Не хочу никому давать копаться в своей голове\n Не хочу, сука\n Боюсь, что раскопают чё-то там', [['Это слова Алишера в интервью Ютуб-каналу «Вписка».\n\nВ клипе к треку скит заканчивается “погружением” в голову моргенштерноголового монстра. Копание в мусорке, раздражённая женщина в бигуди и другие образы представляются при помощи панорамной съёмки — подсказка что мы находимся в голове Моргенштерна и смотрим на мир его глазами.']]), ('Никого кругом, листья падают в бассейн', [['После переезда за границу Моргенштерн приобрёл дом с бассейном в Дубае. Впоследствии он снял несколько клипов и интервью у этого бассейна.\n\n\n\nКадр из клипа на «Double Cup»\n\n\n\n\nВозможно, эту строчку стоит рассматривать как поэтическую аллегорию об одиночестве и застое и отсылку к цитате из романа Великий Гэтсби:\n\nЯ нынче собираюсь спустить воду в бассейне, мистер Гэтсби. Скоро листья начнут опадать, а от них всегда беда с трубами. — Только не сегодня, — ответил Гэтсби. Он с каким-то виноватым видом 

In [16]:
all_song_data.tail(5)

,Year,Rank,Song Title,Artist,Primary Artists,Lyrics,Song URL,Song id,Song stats
6,2022,7,Kill Bill,SZA,"SZA, 0 songs",TranslationsEspañolPortuguêsItalianoTürkçeDeut...,https://genius.com/Sza-kill-bill-lyrics,8616820,"Stats('unreviewed_annotations, concurrents, ho..."
7,2022,8,Feliz Navidad,Jose Feliciano,"José Feliciano, 0 songs","Feliz Navidad Lyrics[Letra de ""Feliz Navidad""]...",https://genius.com/Jose-feliciano-feliz-navida...,623111,"Stats('unreviewed_annotations, hot, pageviews')"
8,2022,9,Anti-Hero,Taylor Swift,"Taylor Swift, 0 songs",TranslationsPortuguêsEspañolTürkçeFrançais中文Ne...,https://genius.com/Taylor-swift-anti-hero-lyrics,8434253,"Stats('unreviewed_annotations, concurrents, ho..."
9,2022,10,Unholy,Sam Smith & Kim Petras,"Sam Smith & Kim Petras, 0 songs",TranslationsFrançaisEspañolNederlandsDeutschال...,https://genius.com/Sam-smith-and-kim-petras-un...,8302862,"Stats('unreviewed_annotations, concurrents, ho..."
10,2022,11,The Christmas Song (Merry Christmas To You),Nat King Cole,"The Nat “King” Cole Trio, 0 songs",The Christmas Song (Merry Christmas to You) (O...,https://genius.com/The-nat-king-cole-trio-the-...,1717793,"Stats('unreviewed_annotations, hot, pageviews')"


In [17]:
all_song_data.to_csv("all_songs_data.csv")
all_song_data.to_json("all_song_data.json", orient='records')

In [ ]:
loaded_song_dataset = pd.read_csv("all_songs_data.csv", index_col=0)

In [ ]:
display(loaded_song_dataset.head())
display(loaded_song_dataset.tail())

In [ ]:
songs_with_lyrics_dataset = loaded_song_dataset.dropna(subset=['Lyrics'])


###

## Understanding lyrics with T5 transformer

* TODO: ADD BLEU/ROUGE evaluation

In [ ]:
MODEL_NAME = 'igorktech/t5-base-en'
SAVE_MODEL_NAME = 't5-base-en-explainer'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

BATCH_SIZE = 32
REPORT_STEPS = 1000
EPOCHS = 1
LEARNING_RATE = 1e-4
TASK_PREFIX = "explain | "
MAX_INPUT = 64
MAX_OUTPUT = 64
# model.config.max_length = MAX_OUTPUT #not mandatory
SAVE_TO_HUB = True

In [ ]:
pairs = []
for a, b in zip(data['train']['short'], data['train']['long']):
    pairs.append([TASK_PREFIX + a, b])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
model.train()
losses = []
for epoch in range(EPOCHS):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / BATCH_SIZE)):
        batch = pairs[i * BATCH_SIZE: (i + 1) * BATCH_SIZE]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding="longest", \
                      max_length=MAX_INPUT, truncation=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding="longest", \
                      max_length=MAX_OUTPUT, truncation=True, ).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % REPORT_STEPS == 0:
            print('step', i, 'loss', np.mean(losses[-REPORT_STEPS:]))

### Save model

In [ ]:
if SAVE_TO_HUB:
    notebook_login()
    model.push_to_hub(SAVE_MODEL_NAME)
    tokenizer.push_to_hub(SAVE_MODEL_NAME)
else:
    model.save_pretrained(SAVE_MODEL_NAME)
    tokenizer.save_pretrained(SAVE_MODEL_NAME)

### Testing model

In [ ]:
model.eval()


def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device)
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [ ]:
print('model: ', answer(TASK_PREFIX + 'What is love?'))